# Compare CPU and GPU image preprocessing in Spark OCR
* Load images from S3
* Preview images
* Recognize text

## Import OCR transformers and utils

In [0]:
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.databricks import display_images
from pyspark.ml import PipelineModel

11:51:09, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

## Define OCR transformers and pipeline
* Transforrm binary data to Image schema using [BinaryToImage](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#binarytoimage). More details about Image Schema [here](https://nlp.johnsnowlabs.com/docs/en/ocr_structures#image-schema).
* Recognize text using [ImageToText](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#imagetotext) transformer.

In [0]:
def pipeline():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        ocr
    ])
    
    return pipeline

11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

In [0]:
def pipelineCPU():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")
    
    # Scale image
    scaler = ImageScaler()\
      .setInputCol("image")\
      .setOutputCol("scaled_image")\
      .setScaleFactor(2.0)
    
    # Binaraze image
    adaptive_thresholding = ImageAdaptiveBinarizer() \
      .setInputCol("scaled_image") \
      .setOutputCol("corrected_image") \
      .setMethod(ThresholdingMethod.OTSU)
    
    # Apply erosion
    erosion = ImageMorphologyOperation() \
      .setKernelShape(KernelShape.SQUARE) \
      .setKernelSize(1) \
      .setOperation("erosion") \
      .setInputCol("corrected_image") \
      .setOutputCol("eroded_image")

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("eroded_image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        scaler,
        adaptive_thresholding,
        erosion,
        ocr
    ])
    
    return pipeline

11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

In [0]:
def pipelineGPU():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")
    
    # Image transformation on GPU
    transformer = GPUImageTransformer() \
      .addScalingTransform(2) \
      .addOtsuTransform() \
      .addErodeTransform(1, 1) \
      .setInputCol("image") \
      .setOutputCol("eroded_image") \

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("eroded_image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        transformer,
        ocr
    ])
    
    return pipeline

11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

## Download images from public S3 bucket to DBFS

In [0]:
%sh
OCR_DIR=/dbfs/tmp/ocr_1
if [ ! -d "$OCR_DIR" ]; then
    mkdir $OCR_DIR
    cd $OCR_DIR
    wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/ocr/datasets/news.2B.0.png.zip
    unzip news.2B.0.png.zip
fi

11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

In [0]:
display(dbutils.fs.ls("dbfs:/tmp/ocr_1/0/"))

path,name,size
dbfs:/tmp/ocr_1/0/.DS_Store,.DS_Store,10244
dbfs:/tmp/ocr_1/0/8200_006.2B.png,8200_006.2B.png,101724
dbfs:/tmp/ocr_1/0/8200_006.2B.tif,8200_006.2B.tif,116158
dbfs:/tmp/ocr_1/0/8200_006.2B.txt,8200_006.2B.txt,3429
dbfs:/tmp/ocr_1/0/8200_006.2B.uzn,8200_006.2B.uzn,177
dbfs:/tmp/ocr_1/0/8200_014.2B.png,8200_014.2B.png,95597
dbfs:/tmp/ocr_1/0/8200_014.2B.tif,8200_014.2B.tif,116116
dbfs:/tmp/ocr_1/0/8200_014.2B.txt,8200_014.2B.txt,3309
dbfs:/tmp/ocr_1/0/8200_014.2B.uzn,8200_014.2B.uzn,87
dbfs:/tmp/ocr_1/0/8200_015.2B.png,8200_015.2B.png,64646


11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0
11:51:10, INFO Received command c on object id p0

## Read images as binary files
from DBFS

In [0]:
images_path = "/tmp/ocr_1/0/*.png"
images_example_df = spark.read.format("binaryFile").load(images_path).cache()
display(images_example_df)

11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0

path modificationTime length content dbfs:/tmp/ocr_1/0/8230_014.2B.png 2022-04-21T15:44:22.000+0000 234628 List(iVBORw0KGgoAAAANSUhEUgAACKAAAAakCAAAAADsJq6YAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOxd6ZrkKq4U95v3f2XuD4N2AXZmdbv6RMyczrLZxC4Fst06AQAAAAAAvAv/97cFAAAAAAAA8ICCAgAAAADA6wA= (truncated), iVBORw0KGgoAAAANSUhEUgAAAGQAAABMCAAAAAC+dk3TAAAWwUlEQVR42u2Zd3QTx7fHd9UlV9xrjAGDTXPoPbRQQkkoocQhP0qoAUJCgADhF8DBVIMxwTRjGxfcZUuWJdmSVazee7VsufdKM8Vl3yqJzEtOfuf9zjvvvb/enLNaaXZnPntn7vfO3REA/Y+V4X95BYD+D8r/Q/4f8r8OGW7rftHR9uLly5eDL/va2rp7u9rb2tt+L+29XXD9yxcvnnfCvzo72tpe/dbmTd+LV+9e/7XDt+19jrtfvv0bS0SsoqTKvwiq4Q4kKjfHmmosRoWoPEtabS7llMSx+CYGU64vTU+VCaqExZWl5Zfy2TzK+a/vFUjMeuHWvGQyS6jQ2Ww2q0FvUltqzO1OyO6klptn6TaKoCCfnSoXMgeZ0q7kLZD4/g+3m/o7W2r6pY3U+hfS5Lw4cq3thk5fN3iaSevQC2VmWQbpeLopaXFymqC9z1Ix95t7RUYuk8N6Jhb3D0KvGuqfDzsh9UeCjyUvXJuw9Xvq1rE5G+58lbBxe+kWSNK77xaRXaYqMNjt9T/re9JuH7nyS8rONvGduoLSi1e4iuWJx/ta15w5volwWSBv73rWu2zzshuU/Ueul1XLyM0DAwP17UlsJ+Tu66db0uYsLY99MPxJYlFM3ImEDV8nfQpJEj+qVcYek965nSTUk6rfPIsr0mxedeb4mR3W12s3fXyEG/rLKRtz8tLwUK+nScnGnnpbyHLxudJKVu9AT/2tvsaqztb8p/1OSI1AaTUIGiqqzFDPs0GbFJJbnjHlUBbmmyt3vzzPtt6JzzdbyMNDivSEhOhpH08aGyPfMOcfd9Ym7I6e/ZHfKP/ZirJNP6eopBUeY+dsTGp9w0jkCpr68we7js6qkjkhJh6/4W+cLyn6kjQ65nsW+2Yqg8x8MAwN0eSH4w9fi3N1J7gSbi2dmkNfFB688xdcBHvzoWPxT8+dxM3ott+M//n+8MuW7qIhaGAKcbDXCRFs7+W/g6Cht9A76P57CGlj4YUpO/OWrL/Ikw60H25uHuId7I+de9rzxMPFSNAHd/FO9icr1S+fzQk4RItx9fMgrIj2MxgFA/0Knfx40m114/5pucMjLixdcePorXMLDyYc+iIl0jwC4bjdSFy9OWfu5zkK6vMh4f4zLbJzLRkfHYk7tvnBTCQCAfq7LEmL1bJOIHwC/Eejjhw8PA9YUynreveaW9VvnTN+9cpx+a9GIPwvbPPoaw/Fnmcf3Dj3vSWZAfPS133x0c/x+xT1Fq7AILh+IjJqD2/IfOPDqPwz7gAITF7xVczavQfcQHwEInrhNj0/AIgh1ven3s/fjfvsxuEtn9QNjkDKzr1Ov194s3oXLffhCfsI5PGU1Dv7F589tjbT8n1SzJpEllYnG88dkt2JPb33g2m7AAQIYL86VZr66ziEy5Rd+64c5fO8gSV3LV2fjUHNe9z3ssuh7uH/InZlRl08H7dy9oyzN6WkjONUYTq9krLSbsipeiTc/uF36wDYFN98bpPxPICc88+4O6c3bxoPhN1fvDIAh5hDlPTVPh/+c4B8QoE/5PcgaGD7AHTs+e+VaT7s2fP2rlhx63jRudO7fyQbu4Z6pyfQ5JyMhV94781AwKZMo5QoS0e7oSec4CTNHIUBCL/8gBhDWDgxJ0fW3zo0BA2/GnRCWPnXT6SUXU1nXLwsWaw03lRePWvJeABlej/1W/j5jg2rP9lF3Jf+Q9M7qIG2jPi4sPjs2ZM7J95aigBR4NbPp3uOiwICE+IXYRHAwnLbDaz3EXqfUTPQDEfS4UFTgxPy7e64Jyvv3ymOXZYZt/gHaDXjwPyzpE0Qy3N0IOrCTzvcJl0vKN+mGh5oqt93mqKO5bMTUw5N9RoL4jzwBBeUy/njAMo/HIkGI/6xcxbokUBKfddUL6EIh4eh3ly5E0I6+MXdeTceEE/NvvvTzF3pi7N3z0zYRITIJzBng7Chri4Rm09+t/Dw3v3mC6tJzOeFyXQj9XL8z8HIgLFLPXCTtp7bjhrvgkIAABYE8Z8lXSBf0LTYmjNuy19BzSePOSHPqnu6Wrt6nnW2dHa2dHU4Pq8kbILIsRj/aVO9EdgA36DTlPNVvJTJj5t7a3QP48qefJaRtRpEY329Zm08QTyO8CGAAAyZHj7nm4OZGzYmN3Zf7YWgV50H1vzjL97VyoMYtczfv/cbnkG0cUGu6ABXABwbMHZrclL90Lqdzc8H35i2FTWor1NeRXuOQ6HW7D7qdSbKgwC67LscNnfdnK93LRg96UwpMztgMzun3MrDf+WEdHUNVL/ubKphde2SLejs6/mj2lxxeAkO9EChZkZOj96845vvR39cpjDU5yxPaXx9lnUTAP1CsZN3fvLl/NUfgIiLZsaV4OWr1wX5btpzKaUtznPjrrc98gnATidk354t5x5FxMQd3vCpbMHPSeV/VFtbzixyB7EEhKd30IRZH61YPmNJzoV/Xq16dcnQ+vNzkycsFCwqxA2BGjsTQEb9aOAGbjq0J3zaigPX7dknA08vib8/Fjz+2gk5qrh34N70q8e3rd0rW9Ay8Q+ZQDbtd+7+aKQ7fhQaEzUGF7wxauqSqV/waXGrkt4kt/X64QJ9gWAMBolBwJP+6fosScmj+V8uPHMpntIWg592bBI8Tdf73zkhqerEe5mjE6R5Vy+L4uyPnXNkVl4KQ2KRATjCOGzoN2N8IoKiY9ZOWnX/QObVQQFDAuAjAl19XNAgchUOAFbUrr1yYN14cNrW1Z9c2gtHTwDwmv5D/9CfwsrzhD9SjFSbs4p6YLy7myvWDQm6uAWtmhTlfi2ZeW3pxqBxny/ub0waGh904kM/EEB67BO5AADmEAbuFz0q4LMNKFcketz2b4NiH1OY/SOQhncdf/azlk6oBYJKdge64EOWRnrBQQr38Zco3N6VBaL7vbYbK4CAD+nQpo8WRQcBQEDEsp1IRxwDA4PDv5k/PTCMAOJDPk/sW07lzZqX/tYJeSJ8lMdgtCoYRVkSSwW/WJpGrpSJIMapcdhxCwjL4EdE+XnO8nRfuW37uiWFcNjjTgRmQKvCo6Nm411OfTbV0wumABgA4T1hy76FaM/V078kn4oJXfVowartLCekTpzHM7TVmXX5DPkQTSVhkEuVeg2U5oby+YCAnhE9NhiN8iK4YkPDwuZv+PBkz1DPT8CnjRGhUd6hSCQ6yB/jhoEZ7gCAcMG6YkYnrBs3a+GudQGLyr6dtPvhn8TYAj2DoEFzX4tDJi8dNdkLJrq7hs+d5uvr5xsCgiiMmxcSgRvvtejO2TVA5I2DyUd8XdDAksOnXBGOiHw4dhlmK+gzarwrxn1lSt6toNAn27ecSXJCctKojMTSWEZ+6suzpY+Tc+5mkR3VT7dF7Vq7IWaRB4jB58zyxzoiB1wQnijAK3Lp7F2PjoS6ABMu3ZgKwi6MOd//pvPs5YXTx368ZPmCH6WpSWMQW4NXTbrghDQrdfUlbH2dUfXWUCdRVLXI6xzVGTsnzPl0/bql3igQWOkZOMo3aOJUhAODQuDHRQT4uDu+j13KDINPYKCw23J66+kJT/ynzd915Fzm4R2jwBCUl9tuJ0T3Smluaql+Z67vrOl9P4CFUa5uU7bsjiL4hk/Aw5EcjUfv+e6A57gAAOmCQ6M8CD4YPBA+q3o0bCLyeEZO+q

11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0
11:55:51, INFO Received command c on object id p0

## Read data from s3 directly using credentials

In [0]:
# ACCESS_KEY = ""
# SECRET_KEY = ""
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", ACCESS_KEY)
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", SECRET_KEY)
# imagesPath = "s3a://dev.johnsnowlabs.com/ocr/datasets/news.2B/0/*.tif"
# imagesExampleDf = spark.read.format("binaryFile").load(imagesPath).cache()
# display(imagesExampleDf)

11:56:05, INFO Received command c on object id p0
11:56:05, INFO Received command c on object id p0
11:56:05, INFO Received command c on object id p0
11:56:05, INFO Received command c on object id p0

## Display count of images

In [0]:
images_example_df.count()

Out[9]: 1811:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0
11:56:06, INFO Received command c on object id p0

## Preview images using _display_images_ function

In [0]:
display_images(BinaryToImage().transform(images_example_df), limit=3)

11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:07, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0
11:56:08, INFO Received command c on object id p0

Origin: dbfs:/tmp/ocr_1/0/8230_014.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr_1/0/8230_022.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr_1/0/8250_007.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1

11:56:09, INFO Received command c on object id p0
11:56:09, INFO Received command c on object id p0
11:56:09, INFO Received command c on object id p0
11:56:09, INFO Received command c on object id p0
11:56:09, INFO Received command c on object id p0

## Run OCR pipeline without preprocessing

In [0]:
result = pipeline().transform(images_example_df.repartition(8)).cache()
result.count()

11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:10, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:11, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:12, INFO Received command c on object id p0
11:56:13, INFO Received command c on object id p0
11:56:13, INFO Received command c on object id p0
11:56:13, INFO Received command c on object id p0
11:56:13, INFO Received command c on object id p0


In [0]:
display(result.select("text", "confidence"))

text,confidence
"Alien sex? Harvard prof believes it Experiences with extraterrestrials real, he says By ANNE THOMPSON Associated Press CAMBRIDGE, Mass. — Dr. John Mack, a Harvard psychiatrist, has never had sex with a space alien, nor does he care to. It could compromise his objectivity. Mack, a Pulitzer Prize winner, firmly believes those people who say they had extremely close encounters with beings from other worlds. And in his new book, “Abduc- tion,” he relates his findings after spending four years studying 120 “experiencers” — people who say they had sexual or reproductive ex- periences with aliens who abducted them and took them aboard space- ships. The 13 case studies in his book include Ed, who remembers an alien woman taking a sperm sample from him when he was in high school; Jerry, who says she gave birth to a human-alien hybrid; and Peter, who tells Mack he had an “alien wife” ina “parallel universe.” “There is a consistent reproduc- tive theme,” Mack said. “Now, do IT mean by that literally there are mil- lions of babies being made? I don’t know. I can’t claim that this is occur- ring literally in our physical dimen- sion of reality.” What Mack does claim is that experiencers are not mentally ill and are not working out the effects of rape, incest or other abuse. Their emotional and psychological state resembles trauma, he said, and trau- ma is the response to something outside the body, not inside the mind. “They don’t want to believe it any more than we do,” he said. “They want me to find them crazy.” Descriptions of abductions are consistent from patient to patient, Mack said. So he believes the exper- iencers, even if he can’t explain exactly what they experienced. One of Mack’s patients, a 37-year- old undercover debunker, told Time magazine that Mack gave her UFO literature to read before he hypno- tized her. “John made it obvious what he wanted to hear,” the patient, Donna Bassett, said. “I provided the an- swers.”” Mack could not be located to respond to Bassett’s allegations. He declined to comment to the magazine about her case. His accomplishments include founding the psychiatry department at the Harvard-affiliated Cambridge Hospital and winning a 1977 Pulitzer for his psychoanalytic biography of T.E. Lawrence (Lawrence of Arabia). Since then, Mack has developed an interest in Eastern philosophy and religion, and believes it helped liber- ate his thinking. “We don’t have room in our cul- ture for this. It’s the elite people, my colleagues, who decide what we're supposed to believe,” he said, “and to them this isn’t supposed to be.” Even Mack’s fellow researchers into the otherworldly think “‘Abduc- tion” is a little, well, out there. Dr. William McCall, a physician in Anaheim, Calif., said that for 10 years he used hypnosis to evaluate exper- iencers. His theory: They’re actually remembering being born. People’s memories of going aboard spacecraft match their hypnosis-evoked memo- ries of passing through the birth canal, he said. Budd Hopkins, a pioneer in UFO abduction research, said that while he agrees with Mack that these people had sexual encounters with aliens, he has reservations about Mack’s counseling of people to see SUSAN WALSH / ASSOCIATED PRESS Dr. John Mack, ¢ Harvard psychi- atrist, says sexual experiences with extraterrestrials are real. the experience as somehow enrich- ing. Hopkins said he sees such ex- periences as neither good nor bad. “T think John is putting a quasi- religious spin on the material that makes the book more vulnerable to criticism,” said Hopkins, a New York artist who has researched and writ- ten about experiencers for 19 years.",94.39163677509015
"More nations are - falling into chaos, anarchy in Africa @ Underlying problems of poverty and hunger, not democratic reforms, fuel the violence and prevent real democracy. By Hubert Kahi DEUTSCHE PRESSE AGENTUR Nairobi — Soldiers and political extremists in Rwanda slaughtered more than 100,000 civilians in three weeks, a

11:56:30, INFO Received command c on object id p0
11:56:30, INFO Received command c on object id p0
11:56:30, INFO Received command c on object id p0
11:56:30, INFO Received command c on object id p0

## CPU image preprocessing

In [0]:
resultCPU = pipelineCPU().transform(images_example_df.repartition(8)).cache()
resultCPU.count()

11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:31, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:32, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:33, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0
11:56:34, INFO Received command c on object id p0


In [0]:
display(resultCPU.select("text", "confidence"))

text,confidence
"Alien sex? Harvard prot believes it Experiences with extraterrestrials real, he says By ANNE THOMPSON Associated Press CAMBRIDGE, Mass. — Dr. John Mack, a Harvard psychiatrist, has never had sex with a space alien, nor does he care to. It could compromise his objectivity. Mack, a Pulitzer Prize winner, firmly believes those people who say they had extremely close encounters with beings from other worlds. And in his new book, “Abduc- tion,” he relates his findings after spending four years studying 120 “experiencers” — people who say they had sexual or reproductive ex- periences with aliens who abducted them and took them aboard space- ships. The 13 case studies in his book include Ed, who remembers an alien woman taking a sperm sample from him when he was in high school; Jerry, who says she gave birth to a human-alien hybrid; and Peter, who tells Mack he had an “alien wife” in a “parallel universe.” “There is a consistent reproduc- tive theme,” Mack said. “Now, do I mean by that literally there are mil- lions of babies being made? I don’t know. I can’t claim that this ts occur- ring literally in our physical dimen- sion of reality.” What Mack does claim is that expenencers are not mentally ill and are not working out the effects of rape, incest or other abuse. Their emotional and psychological state resembles trauma, he said, and trau- ma is the response to something outside the body, not inside the mind. “They don’t want to believe it any more than we do,” he said. “They want me to find them crazy.” Descriptions of abductions are consistent from patient to patient, Mack said. So he believes the exper- iencers, even if he can’t explain exactly what they experienced. One of Mack’s patients, a 37-year- old undercover debunker, told Time magazine that Mack gave her UFO literature to read before he hypno- tized her. “John made it obvious what he wanted to hear,’’ the patient, Donna Bassett, said. “I provided the an- SWers.” Mack could not be located to respond to Bassett’s allegations. He declined to comment to the magazine about her case. His accomplishments include founding the psychiatry department at the Harvard-affiliated Cambridge Hospital and winning a 1977 Pulitzer for his psychoanalytic biography of T.E. Lawrence (Lawrence of Arabia). Since then, Mack has developed an interest in Eastern philosophy and religion, and believes it helped liber- ate his thinking. “We don’t have room in our cul- ture for this. It’s the elite people, my colleagues, who decide what we're supposed to believe,” he said, “and to them this isn’t supposed to be.” Even Mack’s fellow researchers into the otherworldly think ‘‘Abduc- tion” is a little, well, out there. Dr. William McCall, a physician in Anaheim, Calif., said that for 10 years he used hypnosis to evaluate exper- iencers. His theory: They’re actually remembering being born. People’s memories of going aboard spacecraft match their hypnosis-evoked memo- nes of passing through the birth canal, he said. Budd Hopkins, a pioneer m UFO abduction research, said that while he agrees with Mack that these people had sexual encounters with aliens, he has reservations about Mack’s counseling of people to see SUSAN WALSH / ASSOCIATED PRESS Dr. John Mack, a Harvard psychi- atrist, says sexual experiences with extraterrestrials are real. the experience as somehow enrich- ing. Hopkins said he sees such ex- periences as neither good nor bad. “T think John 1s putting a quasi- religious spin on the material that makes the book more vulnerable to criticism,” said Hopkins, a New York artist who has researched and wnit- ten about expenencers for 19 years.",94.34229448106554
"More nations are falling into chaos, anarchy in Africa @ Underlying problems of poverty and hunger, not democratic reforms, fuel the violence and prevent real democracy. By Hubert Kahl DEUTSCHE PRESSE AGENTUR Nairobi — Soldiers and political extremists in Rwanda slaughtered more than 100,000 civilians in three weeks, and the

11:57:26, INFO Received command c on object id p0
11:57:26, INFO Received command c on object id p0
11:57:26, INFO Received command c on object id p0
11:57:26, INFO Received command c on object id p0

## GPU image preprocessing

In [0]:
resultGPU = pipelineGPU().transform(images_example_df.repartition(8)).cache()
resultGPU.count()

11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0
11:57:28, INFO Received command c on object id p0

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2477198187298006> in <module> 
 1 resultGPU = pipelineGPU ( ) . transform ( images_example_df . repartition ( 8 ) ) . cache ( ) 
 ----> 2 resultGPU . count ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in count (self) 
 668 2 
 669 """
 --> 670 return int ( self . _jdf . count ( ) ) 
 671 
 672 def collect ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o967.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 34.0 failed 4 times, most recent failure: Lost task 6.3 in stage 34.0 (TID 110) (10.139.64.11 executor 1): java.lang.NoClassDefFoundError: Could not initialize class net.haesleinhuepf.clij.clearcl.backend.jocl.ClearCLBackendJOCL
	at net.haesleinhuepf.clij.CLIJ.<init>(CLIJ.java:118)
	at net.haesleinhuepf.clij.CLIJ.getInstance(CLIJ.java:172)
	at net.haesleinhuepf.clij2.CLIJ2.getInstance(CLIJ2.java:112)
	at com.johnsnowlabs.ocr.transformers.GPUImageTransformer.$anonfun$transformImage$1(GPUImageTransformer.scala:95)
	at scala.util.Try$.apply(Try.scala:213)
	at com.johnsnowlabs.ocr.transformers.GPUImageTransformer.transformImage(GPUImageTransformer.scala:95)
	at com.johnsnowlabs.ocr.transformers.ImageTransformerBase.$anonfun$localProcess$1(ImageTransformerBase.scala:69)
	at scala.util.Try$.apply(Try.scala:213)
	at com.johnsnowlabs.ocr.transformers.ImageTransformerBase.localProcess(ImageTransformerBase.scala:67)
	at com.johnsnowlabs.ocr.transformers.ImageTransformerBase.$anonfun$transformUDF$1(ImageTransformerBase.scala:62)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.generate_doConsume_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.generate_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:757)
	at org.apache.spark.sql.execution.columnar.DefaultCachedBatchSerializer$$anon$1.hasNext(InMemoryRelation.scala:120)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.storage.memory.MemoryStore.putIterator(MemoryStore.scala:223)
	at org.apache.spark.storage.memory.MemoryStore.putIteratorAsValues(MemoryStore.scala:302)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$1(BlockManager.scala:1430)
	at org.apache.spark.storage.BlockManager.org$apache$spark$storage$BlockManager$$doPut(BlockManager.scala:1357)
	at org.apache.spark.storage.BlockManager.doPutIterator(BlockManager.scala:1421)
	at org.apache.spark.storage.BlockManager.getOrElseUpdate(BlockManager.scala:1240)
	at org.apache.spark.rdd.RDD.getOrCompute(RDD.scala:426)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:377)
	at org.apac

In [0]:
display(resultGPU.select("text", "confidence"))